# Fake News Prediction Model

In [43]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Akash.Kushwaha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [22]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Data collection and pre-processing

In [10]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [23]:
df.shape

(20800, 5)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [26]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [27]:
# replacing null values with empty string

df = df.fillna("")

In [28]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [143]:
# we're going to make predictions on title and author column

# df['content'] = df['author'] + " "+df['title'] 
df['content'] = df['title'] + " " + df['text']

In [144]:
df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired Why the Trut...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...


In [145]:
X = df.drop('label',axis =1)
y = df['label']

In [146]:
X.head()

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Why the Truth Might Get You Fired Why the Trut...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Iranian woman jailed for fictional unpublished...


In [147]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

# Stemming - process of reducing a word to its root word.

In [148]:
port_stem = PorterStemmer()

In [149]:
def stemming(context):
    steemed_context = re.sub('[^a-zA-Z]',' ', context)
    steemed_context = steemed_context.lower()
    steemed_context = steemed_context.split()
    steemed_context = [port_stem.stem(word) for word in steemed_context if not word in stopwords.words('english')]
    steemed_context = ' '.join(steemed_context)
    return steemed_context

In [150]:
df['content'] = df['content'].apply(stemming)

In [151]:
df['content'].head()

0    hous dem aid even see comey letter jason chaff...
1    flynn hillari clinton big woman campu breitbar...
2    truth might get fire truth might get fire octo...
3    civilian kill singl us airstrik identifi video...
4    iranian woman jail fiction unpublish stori wom...
Name: content, dtype: object

In [152]:
X = df['content']
Y = df['label']

### converting the textualo values into numerical

In [153]:
vectorize = TfidfVectorizer()
vectorize.fit(X)

X = vectorize.transform(X)



In [154]:
# X = vectorize.fit_transform(X)

In [155]:
print(X)

  (0, 109030)	0.04990609083545967
  (0, 108977)	0.019354676010992878
  (0, 108029)	0.04485544512214073
  (0, 108025)	0.09621662243119276
  (0, 107982)	0.038159282021560374
  (0, 107945)	0.01147813275743077
  (0, 107301)	0.01735255230119808
  (0, 106492)	0.017366145975473413
  (0, 106404)	0.012739399935297378
  (0, 106318)	0.029584577574600923
  (0, 106240)	0.013058991831767156
  (0, 106040)	0.011950783411487302
  (0, 105194)	0.02611855072825209
  (0, 105158)	0.03177277544269941
  (0, 104155)	0.02186410089725434
  (0, 102749)	0.06644108800122836
  (0, 102063)	0.03365344189752555
  (0, 101812)	0.01664554013312925
  (0, 101044)	0.03865106128681813
  (0, 100406)	0.011250984936024663
  (0, 100396)	0.04386167773803338
  (0, 100343)	0.1380903901680028
  (0, 100195)	0.07239396389315284
  (0, 98910)	0.04004998656481277
  (0, 98345)	0.027532903988470952
  :	:
  (20799, 7431)	0.01069212114450089
  (20799, 7107)	0.02831718153331316
  (20799, 6814)	0.04021811963304997
  (20799, 6776)	0.025500790292

In [156]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [157]:
print(X.shape)
print(x_train.shape)
print(x_test.shape)

(20800, 110766)
(16640, 110766)
(4160, 110766)


In [158]:
# training the modef

In [159]:
model = LogisticRegression()

In [160]:
model.fit(x_train,y_train)

LogisticRegression()

In [161]:
# evaluation of model

x_train_pred = model.predict(x_train)
accuracy_train = accuracy_score(x_train_pred,y_train)

In [162]:
print("Accuracy on train data: ",accuracy_train)

Accuracy on train data:  0.9760216346153846


In [163]:
x_test_pred = model.predict(x_test)
accuracy_test = accuracy_score(x_test_pred,y_test)
print("Accuracy on test data: ",accuracy_test)

Accuracy on test data:  0.9483173076923077


### Making a predictive model

In [164]:


x_new = x_test[10]

prediction = model.predict(x_new)
print(prediction)

[0]


In [165]:
print(x_test.dtype)

float64
